In [8]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32


class Motor(ExampleRobot):
    
    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'duty cm', 'set duty cm', 'tacho cm', 'duty cm' ]
        spec["rollover"] = 1400
        spec["args"] = { 
                "title": f"Motor Speed",
                "plot_width": 1200,
                "plot_height": 700,
            }
        return spec
    
    async def update_plot(self):
        return [
            await self.get('DUTY_CM'),
            await self.get('DUTY_CM'),
            await self.get('TACHO_CM'),
            await self.get('DUTY_CM'),
        ]
        
async def run():
    async with Motor(enable_remote=False) as robot:
        await robot.set('FS', 10)
        await robot.set('PWM_FREQ', 10_000)
        await robot.start('no_control')
        N = 40
        span = 60
        for cps in np.linspace(-span, span, N):
            await robot.set('DUTY_CM', cps)
            await asyncio.sleep(0.2)

try:
    asyncio.run(run())
except KeyboardInterrupt:
    print("so long")

Comm aenter start _repl
comm._repl exec_no_follow ... from robot import Robot
Robot(1000000)

connect /dev/ttyAMA1
Comm aenter connect to AMA2
Comm aenter start cmd_response
*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     
Comm aenter DONE


In [ ]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32


class Motor(ExampleRobot):
    
    BLE_PID = 'TACH_CM'
    BLE_X   = 'DUTY_CM'
    BLE_Y   = 'DUTY_DIFF'
    
    async def ble_event(self, code: str, value: float):
        # print(f"ble_event: {code} {value:8.2f}")
        if code == 'x':
            cm = await self.get(BLE_Y)
            df = abs(100-abs(cm))
            await self.set(BLE_X, df*value)
            return
        if code == 'y':
            await self.set(BLE_Y, 80*value)
            return
        if value < 0:
            value = 0
        if code == '1':
            await self.set_pid(PID, 'KP', 0.1*value)
        if code == '2':
            await self.set_pid(PID, 'KI', 0.1*value)
        if code == '3':
            await self.set_pid(PID, 'KD', 0.1*value)
        if code in [ '1', '2', '3' ]:
            kp = await self.get_pid(PID, 'KP', verify=True)
            ki = await self.get_pid(PID, 'KI', verify=True)
            kd = await self.get_pid(PID, 'KD', verify=True)
            print(f"PID: {kp:8.2f} {ki:8.2f} {kd:8.2f}")                  
        await super().ble_event(code, value)

    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 't', 'set tacho cm', 'tacho cm', 'duty cm' ]
        spec["rollover"] = 1400
        spec["args"] = { 
                "title": f"PID Motor Speed",
                "plot_width": 1200,
                "plot_height": 700,
            }
        return spec
    
    async def update_plot(self):
        return [
            await self.get('K') / await self.get('FS'),
            await self.get('DUTY_CM'),
            await self.get('TACHO_CM'),
            await self.get('DUTY_CM'),
        ]
        
async def run():
    async with Motor(enable_remote=False) as robot:
        await robot.set('FS', 10)
        await robot.set('PWM_FREQ', 10_000)
        await robot.set('DUTY_DEAD', 0)
        await robot.set_pid(PID, 'U_MIN', -100)
        await robot.set_pid(PID, 'U_MAX',  100)
        await set_kp(robot, 0.5)
        await robot.start('speed_control')
        while not robot.stop:
            await asyncio.sleep(0.5)
        print("done")


try:
    asyncio.run(run())
except KeyboardInterrupt:
    print("so long")